In [30]:
import json
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer 
from gensim.models.keyedvectors import KeyedVectors
from gensim.models.doc2vec import TaggedDocument
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec
import multiprocessing
import time

cores = multiprocessing.cpu_count()

# import gensim
# W2V_PATH="GoogleNews-vectors-negative300.bin.gz"
# model_w2v = gensim.models.KeyedVectors.load_word2vec_format(W2V_PATH, binary=True)

In [2]:
caption_comments_replies = open("FaceBook Scrapper/Dataset.json")

In [3]:
caption_comments_replies = json.load(caption_comments_replies)

In [4]:
def tokenization(text):
    tokens = word_tokenize(text)
    return tokens

In [5]:
def Capitalization(text):
    text=text.lower()
    return text

In [6]:
def remove_specail_chars(text):
    text=' '.join(text)
    tokenizer=nltk.RegexpTokenizer(r"\w+")
    text = tokenizer.tokenize(text)
    return text

In [7]:
def text_cleaner(text):
    rules = [
        {r'>\s+': u'>'},  # remove spaces after a tag opens or closes
        {r'\s+': u' '},  # replace consecutive spaces
        {r'\s*<br\s*/?>\s*': u'\n'},  # newline after a <br>
        {r'</(div)\s*>\s*': u'\n'},  # newline after </p> and </div> and <h1/>...
        {r'</(p|h\d)\s*>\s*': u'\n\n'},  # newline after </p> and </div> and <h1/>...
        {r'<head>.*<\s*(/head|body)[^>]*>': u''},  # remove <head> to </head>
        {r'<a\s+href="([^"]+)"[^>]*>.*</a>': r'\1'},  # show links instead of texts
        {r'[ \t]*<[^<]*?/?>': u''},  # remove remaining tags
        {r'^\s+': u''}  # remove spaces at the beginning
    ]
    for rule in rules:
        for (k, v) in rule.items():
            regex = re.compile(k)
            text = regex.sub(v, text)
            text=re.sub('https://[^ ]+','<link>',text)
            text=re.sub('http://[^ ]+','<link>',text)
            text=re.sub('@[^ ]+','<username>',text)
            text=re.sub('#[^ ]+','<hashtag>',text)
    text = text.rstrip()
    return Capitalization(text)

In [8]:
def Lemmatizing(filtered_sentence):
    
    filtered_sentence=remove_specail_chars(filtered_sentence)
    lemmatizer = WordNetLemmatizer()
    
    stemmed_Lemmatized_text=[]

    for word in filtered_sentence:
        word=lemmatizer.lemmatize(word)
        stemmed_Lemmatized_text.append(word)
        
    return stemmed_Lemmatized_text

In [9]:
def remove_stopwords(text):

    stop_words = set(stopwords.words('english'))
    
    cleaned_text=text_cleaner(text)
    
    word_tokens = tokenization(cleaned_text)

    filtered_sentence = [w for w in word_tokens if not w in stop_words]

    filtered_sentence = []

    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
            
    filtered_sentence=Lemmatizing(filtered_sentence)
    
    filtered_sentenc=remove_specail_chars(filtered_sentence)

    return filtered_sentence
    

In [10]:
def Cosine_Similarity(document_1,document_2,N):
    
    numerator = 0
    
    mod_x = 0
    mod_y = 0
    
    for i in range(N):
        numerator += document_1[i]*document_2[i]
        mod_x += document_1[i]**2
        mod_y += document_2[i]**2
    
    denominator = mod_x**(1/2) * mod_y**(1/2)
    
    if denominator==0:
        return 0

    Cosine_similarity = numerator/denominator
    
    return Cosine_similarity

In [11]:
def calculate_cosine_similarity(caption_index,documents):
    

    tfidfvectoriser = TfidfVectorizer()
    tfidf_vectors = tfidfvectoriser.fit_transform(documents)

    vector = tfidf_vectors.toarray()

    cosine_similarity = {}
    
    for i in range(len(vector)):
        
        cosine_similarity[i] = Cosine_Similarity(vector[0],vector[i],len(vector[0]))

    document_cosine_similarity[caption_index] = dict(sorted(cosine_similarity.items(), 
                                                            key = lambda item: item[1],reverse = True))

In [12]:
def get_captions_and_comments(index):
    
    caption = caption_comments_replies[index]['content']
    comments = caption_comments_replies[index]['comments_and_replies']
    
    return caption,comments

In [13]:
document_cosine_similarity = {}

all_captions_and_comments = []

def iterate_all_captions_and_comments():
    
    for index in range(len(caption_comments_replies)):
        
        caption,comments = get_captions_and_comments(index)
        
        all_comments_and_relpies=[]

        for i in comments:
            
            all_comments_and_relpies.append(i['comment'])
            
            for j in i['replies']:
                
                all_comments_and_relpies.extend((list(j.values())))
        
        documents=[]
        
        filtered_caption=remove_stopwords(caption)
        
        documents.append(' '.join(filtered_caption))
        
        filtered_comments=[]
        
        for each_comment in all_comments_and_relpies:
            filtered_comments.append(remove_stopwords(each_comment))
        
        for each_filtered_comment in filtered_comments:
            documents.append(' '.join(each_filtered_comment))
            
        
        calculate_cosine_similarity(index,documents)
        
        all_captions_and_comments.append(documents)
        
        print(index," Done")
        
        
iterate_all_captions_and_comments()

0  Done
1  Done
2  Done
3  Done
4  Done
5  Done
6  Done
7  Done
8  Done
9  Done
10  Done
11  Done
12  Done
13  Done
14  Done
15  Done
16  Done
17  Done
18  Done
19  Done
20  Done
21  Done
22  Done
23  Done
24  Done
25  Done
26  Done
27  Done
28  Done
29  Done
30  Done
31  Done
32  Done
33  Done
34  Done
35  Done
36  Done
37  Done
38  Done
39  Done
40  Done
41  Done
42  Done
43  Done
44  Done
45  Done
46  Done
47  Done
48  Done
49  Done
50  Done
51  Done
52  Done


In [14]:
def print_result():
    for index,caption_and_comments in document_cosine_similarity.items():
        
        print("Caption : {}\n".format(all_captions_and_comments[index][0]))
        
        for caption_index in list(caption_and_comments.keys()):
            
            print("{} : {}\n".format(all_captions_and_comments[index][caption_index],
                                   document_cosine_similarity[index][caption_index]))
        print('\n******************************************************************\n')

In [62]:
words = []

all_captions_and_comments_dict = {}

for i in range(len(all_captions_and_comments)):
    
    temp_list = []
    
    for j in all_captions_and_comments[i]:
        
        words.append(j.split())
        
        temp_list.append(j.split())
        
    all_captions_and_comments_dict[i] = temp_list

In [28]:
def train_w2v(w2v_df):
    
    w2v_model = Word2Vec(min_count = 1,
                         window = 4,
                         size = 300, 
                         alpha = 0.03, 
                         min_alpha = 0.0007, 
                         sg = 1,
                         workers = cores-1)
    
    w2v_model.build_vocab(w2v_df, progress_per = 10000)
    w2v_model.train(w2v_df, total_examples = w2v_model.corpus_count, epochs = 100, report_delay = 1)
    return w2v_model

In [31]:
start_time = time.time()

w2v_model = train_w2v(words)

end_time = time.time()

print("Execution Time in seconds : {}".format(end_time-start_time))

Execution Time in seconds : 57.589138984680176


In [32]:
print(w2v_model.wv.most_similar('ten'))

[('noise', 0.5119937658309937), ('assuredly', 0.5000365972518921), ('boringness', 0.4954816997051239), ('akin', 0.495316743850708), ('mutilated', 0.48628294467926025), ('boredom', 0.4857534170150757), ('nine', 0.4738732576370239), ('hol', 0.4715283215045929), ('lifeguard', 0.46482184529304504), ('craziest', 0.45724788308143616)]


In [33]:
w2v_model.wv.get_vector('clothing')

array([-2.12201718e-02, -4.83483039e-02, -9.51807201e-02,  2.64880389e-01,
        3.03841174e-01, -1.32370427e-01,  3.85881364e-01,  7.51205087e-01,
       -1.89196184e-01, -7.82734677e-02,  2.76383787e-01, -2.08512962e-01,
       -5.67459583e-01,  3.21848929e-01,  6.24212101e-02,  9.69878286e-02,
        3.43349218e-01, -5.44821024e-01, -1.20351113e-01, -1.70302778e-01,
        1.58805907e-01,  7.87084639e-01, -4.47963387e-01, -2.03970447e-01,
       -4.06556129e-01, -1.79679736e-01,  8.99469674e-01,  5.17007709e-01,
       -1.30802423e-01, -5.26438355e-02, -4.10433203e-01, -7.60870501e-02,
       -2.98770100e-01, -2.50768065e-01,  1.49919419e-02, -1.31425440e-01,
       -4.27628696e-01, -6.47646308e-01,  3.18104446e-01, -7.17146218e-01,
        2.59898305e-01, -2.37030238e-01,  3.96885455e-01, -5.14332116e-01,
        2.19229460e-01, -2.79967695e-01, -5.59937179e-01,  3.38063061e-01,
       -9.21291590e-01, -2.99841076e-01,  9.17463377e-03,  1.57968879e-01,
       -3.59685361e-01,  

In [68]:
index = 0

for i in range(len(all_captions_and_comments)-52):
    
    caption = all_captions_and_comments_dict[i][0]
    
    for comment in all_captions_and_comments_dict[i]:
        
        if comment!=[]:
            
            print(comment,end=" : ")
            
            print(w2v_model.wv.n_similarity(caption,comment))
    
    print("\n\n")

['nutrition', 'food', 'dropped', 'significantly', 'past', '70', 'year', 'happening', 'bbc', 'com', 'modern', 'food', 'regain', 'nutrient'] : 1.0
['glad', 'husband', 'chose', 'live', 'part', 'family', 'farm', 'harvest', 'hay', 'chicken', 'turkey', 'get', 'kitchen', 'scrap', 'coffee', 'ground', 'egg', 'shell', 'composted', 'manure', 'go', 'garden', 'raise', 'steer', 'every', 'year', 'get', 'calf', 'born', 'milk', 'make', 'cheese', 'garden', 'grow', 'best', 'tasting', 'tomato', 'ton', 'work', 'husband', 'professional', 'career', 'investment', 'tractor', 'u', 'worth', 'quality', 'life', 'food', 'family'] : 0.6274495
['top', 'soil', 'planet', 'depleted', 'amongst', 'thing', 'wish', 'could', 'buried', 'directly', 'ground', 'wooden', 'box', 'full', 'chemical', 'die', 'much', 'like', 'compost', 'farm'] : 0.58257043
['maribela', 'ruano'] : 0.2540232
['imagine', 'anybody', 'say', 'gmo'] : 0.42024666
['jeff', 'davis', 'hi', 'jeff'] : 0.24653697
['soil', 'depleted'] : 0.44483757
['s', 'way', 'boom

In [ ]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(documents_df.documents_cleaned)
tokenized_documents=tokenizer.texts_to_sequences(documents_df.documents_cleaned)
tokenized_paded_documents=pad_sequences(tokenized_documents,maxlen=64,padding='post')
vocab_size=len(tokenizer.word_index)+1

# reading Glove word embeddings into a dictionary with "word" as key and values as word vectors
embeddings_index = dict()

with open('glove.6B.100d.txt','r',encoding='utf-8') as file:
    for line in file:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

In [ ]:
import numpy as np


class DocSim:
    def __init__(self, w2v_model, stopwords=None):
        self.w2v_model = w2v_model
        self.stopwords = stopwords if stopwords is not None else []

    def vectorize(self, doc: str) -> np.ndarray:
        doc = doc.lower()
        words = [w for w in doc.split(" ") if w not in self.stopwords]
        word_vecs = []
        for word in words:
            try:
                vec = self.w2v_model[word]
                word_vecs.append(vec)
            except KeyError:
                # Ignore, if the word doesn't exist in the vocabulary
                pass

        # Assuming that document vector is the mean of all the word vectors
        # PS: There are other & better ways to do it.
        vector = np.mean(word_vecs, axis=0)
        return vector

    def _cosine_sim(self, vecA, vecB):
        """Find the cosine similarity distance between two vectors."""
        csim = np.dot(vecA, vecB) / (np.linalg.norm(vecA) * np.linalg.norm(vecB))
        if np.isnan(np.sum(csim)):
            return 0
        return csim

    def calculate_similarity(self, source_doc, target_docs=None, threshold=0):
        """Calculates & returns similarity scores between given source document & all
        the target documents."""
        if not target_docs:
            return []

        if isinstance(target_docs, str):
            target_docs = [target_docs]

        source_vec = self.vectorize(source_doc)
        results = []
        for doc in target_docs:
            target_vec = self.vectorize(doc)
            sim_score = self._cosine_sim(source_vec, target_vec)
            if sim_score > threshold:
                results.append({"score": sim_score, "doc": doc})
            # Sort results by score in desc order
            results.sort(key=lambda k: k["score"], reverse=True)

        return results

In [ ]:
ds = DocSim(model_w2v)

# ds = DocSim(embeddings_index)

source_doc = fst_caption
target_docs = documents[1:]

sim_scores = ds.calculate_similarity(source_doc, target_docs)

print(fst_caption)
print(sim_scores)